In [1]:
!pip install konlpy

In [ ]:
import json
import re
from konlpy.tag import Okt  # Okt - 한글 품사 태깅을 위한 모듈
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud

In [ ]:
inputFileName = ''
data = json.loads(open(inputFileName+'.json', 'r', enconding='utf8').read()) # json 파일을 읽어서 객체에 저장
data # 출력하여 내용 확인

In [ ]:
# 'message' 키의 값(뉴스 본문 내용)에서 문자나 숫자가 아닌것 (r'[^\w])은 
# 공백으로 치환(re.sub())하여 제거하면서 연결하여 전체를 하나의 문자열로 구성 
message = ''
for item in data:
    if 'message' in item.keys():
        message = message + re.sub(r'[^\w]', ' ', item['message']) + ''
message #출력하여 내용 확인

In [ ]:
# 품사 태깅 패키지인 Okt를 사용하여 명사만 추출해 (nlp.nouns()) message_N에 저장
nlp = Okt()
message_N = nlp.nouns(message)
message_N # 


In [ ]:
# Counter() 함수를 사용하여 단어별 출현 횟수를 계산
count = Counter(message_N)
count #

In [ ]:
# 출현 횟수가 많은 상위 80개 단어 중에서 길이가 1보다 큰 것만 word_count에 저장후 출력
word_count = dict()
for tag, counts in count.most_common(80):
    if(len(str(tag)) > 1):
        word_count[tag] = counts
        print("%s : %d" % (tag, counts))

In [ ]:
# 히스토그램에 레이블을 한글로 표시하기 위해 한글 폰트인 맑은고딕체 (malgun.ttf)를 설정 (matplotlib.rc())
font_path = "c:/Windows/fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname = font_path).get_name()
matplotlib.rc('font', family = font_name)

In [ ]:
# 히스토그램 생성
plt.figure(figsize = (12, 5))
plt.xlabel('키워드')
plt.ylabel('빈도수')
plt.grid(True)
sorted_Keys = sorted(word_count, key = word_count.get, reverse = True)
sorted_Values = sorted(word_count.values(), reverse = True)
plt.bar(range(len(word_count)), sorted_Values, align = 'center')
plt.xticks(range(len(word_count)), list(sorted_Keys), rotation = '75')
plt.show()

In [ ]:
# 워드클라우드 객체를 생성후 word_count에서 단어별 빈도수를 계산하여 (wc.generate_from_frequencies())
# cloud 객체에 저장하고, 워드클라우드를 생성 plt.imshow()
wc = WordCloud(font_path, background_color = 'ivory', width = 800, height = 600)
cloud = wc.generate_from_frequencies(word_count)
plt.figure(figsize = (8,8))
plt.imshow(cloud)
plt.axis('off')
plt.show()


In [ ]:
cloud.to_file(inputFileName + '_cloud.jpg')